In [1]:
import yfinance as yf
import requests
import pandas as pd
import datetime as dt
import time
from datetime import timedelta
from prod.model import *
from functools import reduce
from yahoo_fin import stock_info as si
import numpy as np
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials
import pygsheets


In [2]:
# inversion inicial en dolares
#initial_amount=5249000/777.18
initial_amount=5000000/777.18

# ponderacion del portafolio
ponderacion={'ETHO':4.06/100,
'ESGV':4.12/100,
'VT':12.16/100,
'XLK':15.16/100,
'VTI':19.61/100,
'IVV':11.42/100,
'QQQ':15.30/100,
'ESGU':4.79/100,
'SMH':6.72/100,
'ARKW':2.30/100,
'ARKK':2.13/100
}

#'OTROS':2.23/100

# portafolio en el que se invierte
tickers=["ESGV","ETHO","VT","XLK","VTI","IVV","QQQ","SMH","ARKW","ARKK","ESGU"]

In [3]:
dataframes=[]
for i in tickers:
       
    # Generamos el diccionario pada cada ticker
    string="{i}=ticker('{i}')".format(i=i)
    exec(string)
    
    string2="df_{i}={i}.getLatest()".format(i=i)
    exec(string2)
    
    string3="df_{i}['ticker']='{i}'".format(i=i)
    exec(string3)
    
    string4="dataframes.append(df_{i})".format(i=i)
    exec(string4)

# Concatenamos todos los dataframes en solo uno
day_display=reduce(lambda x, y: pd.concat([x, y]), dataframes)
day_display

08/13/2020
Creating ticker ...
ESGV was created, date:08/13/2020
08/13/2020
Creating ticker ...
ETHO was created, date:08/13/2020
08/13/2020
Creating ticker ...
VT was created, date:08/13/2020
08/13/2020
Creating ticker ...
XLK was created, date:08/13/2020
08/13/2020
Creating ticker ...
VTI was created, date:08/13/2020
08/13/2020
Creating ticker ...
IVV was created, date:08/13/2020
08/13/2020
Creating ticker ...
QQQ was created, date:08/13/2020
08/13/2020
Creating ticker ...
SMH was created, date:08/13/2020
08/13/2020
Creating ticker ...
ARKW was created, date:08/13/2020
08/13/2020
Creating ticker ...
ARKK was created, date:08/13/2020
08/13/2020
Creating ticker ...
ESGU was created, date:08/13/2020


,date,open,high,low,last_price,change,price_change,ticker
0,2020-08-13,61.840000,61.985001,61.529999,61.709999,-0.210222,-0.130001,ESGV
0,2020-08-13,44.810001,44.931198,44.459999,44.689899,-0.268025,-0.120102,ETHO
0,2020-08-13,81.699997,81.980003,81.410004,81.650002,-0.061194,-0.049995,VT
0,2020-08-13,114.070000,114.910004,113.550003,113.959999,-0.096433,-0.110001,XLK
0,2020-08-13,170.789993,171.744995,170.485001,171.029999,0.140527,0.240005,VTI
0,2020-08-13,338.000000,339.614990,337.179993,338.220001,0.065089,0.220001,IVV
0,2020-08-13,272.640015,274.829987,271.519989,272.480011,-0.058687,-0.160004,QQQ
0,2020-08-13,172.410004,172.589996,170.100006,170.860001,-0.899022,-1.550003,SMH
0,2020-08-13,99.150002,100.345001,98.922401,99.559998,0.413511,0.409996,ARKW
0,2020-08-13,84.510002,86.389999,84.510002,85.760002,1.479115,1.250000,ARKK


In [4]:
# agregamos una columna de ponderacion de cada instrumento
day_display["pond"]=day_display["ticker"].apply(lambda x: ponderacion.get(x))

# Extraemos los montos que no tuvieron pnderacion
not_considered_per=1-sum(day_display["pond"])

# Guardamos lo que no se invirtio.. no se considera en la estimacion
rest=initial_amount*not_considered_per
invested=initial_amount-rest

# Agregamos columnas de cantidad inicial invertida y no invertida independiente del fondo
day_display["initial_invested_usd"]=initial_amount
day_display["real_invested"]=invested
day_display["not_invested"]=rest

# Calculamos lo que se invirtio en cada instrumento
day_display["invested"]=day_display["pond"].apply(lambda x: x*initial_amount)

# Calculamos las ganancias o perdidas
day_display["profit"]=(day_display["change"]/100)*(day_display["invested"])

# Calculamos el nuevo valor de la cantidad invertida en el instrumento 
day_display["estimated"]=day_display["invested"]+day_display["profit"]

# Agregamos una columna con el resultado final de lo invertido
day_display['total_result']=sum(day_display["estimated"])-sum(day_display["invested"])

In [5]:
day_display

,date,open,high,low,last_price,change,price_change,ticker,pond,initial_invested_usd,real_invested,not_invested,invested,profit,estimated,total_result
0,2020-08-13,61.840000,61.985001,61.529999,61.709999,-0.210222,-0.130001,ESGV,0.0412,6433.516045,6290.048637,143.467408,265.060861,-0.557215,264.503646,-2.210849
0,2020-08-13,44.810001,44.931198,44.459999,44.689899,-0.268025,-0.120102,ETHO,0.0406,6433.516045,6290.048637,143.467408,261.200751,-0.700083,260.500669,-2.210849
0,2020-08-13,81.699997,81.980003,81.410004,81.650002,-0.061194,-0.049995,VT,0.1216,6433.516045,6290.048637,143.467408,782.315551,-0.478729,781.836822,-2.210849
0,2020-08-13,114.070000,114.910004,113.550003,113.959999,-0.096433,-0.110001,XLK,0.1516,6433.516045,6290.048637,143.467408,975.321032,-0.940527,974.380506,-2.210849
0,2020-08-13,170.789993,171.744995,170.485001,171.029999,0.140527,0.240005,VTI,0.1961,6433.516045,6290.048637,143.467408,1261.612496,1.772902,1263.385399,-2.210849
0,2020-08-13,338.000000,339.614990,337.179993,338.220001,0.065089,0.220001,IVV,0.1142,6433.516045,6290.048637,143.467408,734.707532,0.478215,735.185747,-2.210849
0,2020-08-13,272.640015,274.829987,271.519989,272.480011,-0.058687,-0.160004,QQQ,0.1530,6433.516045,6290.048637,143.467408,984.327955,-0.577670,983.750284,-2.210849
0,2020-08-13,172.410004,172.589996,170.100006,170.860001,-0.899022,-1.550003,SMH,0.0672,6433.516045,6290.048637,143.467408,432.332278,-3.886760,428.445518,-2.210849
0,2020-08-13,99.150002,100.345001,98.922401,99.559998,0.413511,0.409996,ARKW,0.0230,6433.516045,6290.048637,143.467408,147.970869,0.611876,148.582745,-2.210849
0,2020-08-13,84.510002,86.389999,84.510002,85.760002,1.479115,1.250000,ARKK,0.0213,6433.516045,6290.048637,143.467408,137.033892,2.026889,139.060780,-2.210849


In [6]:
def save_gsheets(page,dataframe):
    gc = pygsheets.authorize(service_file='key.json')

    #open the google spreadsheet 
    sh = gc.open_by_url('https://docs.google.com/spreadsheets/d/1z7zzHgM6t4ySUkPZe5BdFsBvTD6RxsfSdKEnqWU1a2Q/edit#gid=67598709')

    #select the first sheet 
    wks = sh[page]

    #update the first sheet with df, starting at cell B2. 
    wks.set_dataframe(dataframe, 'A1')

In [7]:
save_gsheets(0,day_display)

In [8]:
invested

6290.048637381302

In [9]:
initial_amount

6433.516045189017